In [1]:
# from roboflow import Roboflow
# rf = Roboflow(api_key="T7uEFXRKxnlKwmjBGbBD")
# project = rf.workspace("kolzek").project("face-anti-spoofing-ezpqo")
# version = project.version(4)
# dataset = version.download("yolov11")
                

In [2]:
import torch
torch.cuda.is_available()

True

In [8]:
# yolo_opencv_face_detect.py
# Requires: pip install ultralytics opencv-python
# Replace 'yolov8n-face.pt' with the path to your face-specific YOLO weights.
from ultralytics import YOLO
import cv2
import numpy as np

# Load YOLO model (use a face-detection weight; change path if needed)
# model = YOLO('yolov11n-face.pt')

# model = model.train(data="Face-Anti-Spoofing-4/data.yaml",epochs=50,batch=16,imgsz=320,device=0,lr0=0.003,patience=10)  # Set model to training mode
model = YOLO("runs/detect/train5/weights/best.pt")
result = model.predict("Test.jpg", show=True,save=True)  # Inference on an image file

WARNING Environment does not support cv2.imshow() or PIL Image.show()
OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


image 1/1 c:\Users\gifly\Desktop\Playground\User-Authentication-by-Facial-Recognition\Test.jpg: 192x320 1 fake, 42.1ms
Speed: 2.5ms preprocess, 42.1ms inference, 2.5ms postprocess per image at shape (1, 3, 192, 320)
Results saved to C:\Users\gifly\Desktop\Playground\User-Authentication-by-Facial-Recognition\runs\detect\predict


In [5]:
# Open default camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise SystemExit("Cannot open camera")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Run model on the BGR frame (Ultralytics accepts numpy arrays)
        results = model(frame)[0]  # results is a single Result object

        # Extract boxes, confidences, classes
        if results.boxes:
            xyxy = results.boxes.xyxy.cpu().numpy()    # shape (N,4)
            confs = results.boxes.conf.cpu().numpy()   # shape (N,)
            clss = results.boxes.cls.cpu().numpy()     # shape (N,)
            names = results.names                       # dict/list of class names

            for (x1, y1, x2, y2), conf, cls in zip(xyxy, confs, clss):
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                label = names[int(cls)] if int(cls) in names else str(int(cls))
                text = f"{label} {conf:.2f}"
                # Draw box and label
                # cv2.rectangle(frame, (x1-50, y1-50), (x2+50, y2+50), (0, 255, 0), 3)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                cv2.putText(frame, text, (x1, y1 - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Show frame
        cv2.imshow("YOLO Face Detection", frame)

        # Quit on 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    cap.release()
    cv2.destroyAllWindows()


0: 256x320 (no detections), 10.9ms
Speed: 1.7ms preprocess, 10.9ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 320)


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
